In [ ]:
# Loading the required Libraries.
import _pickle as pickle
import pandas as pd
import numpy as np
from sklearn import preprocessing
from typing import Optional, TextIO, Type, Tuple


In [ ]:
pwd()

In [ ]:
Training_Data = 'Dota_2.vp'

In [ ]:
with open(Training_Data, 'rb') as File:
    D = pickle.Unpickler(File)
    Data = D.load()

In [ ]:
len(Data[0])

In [ ]:
# Team 1’s hero IDs of 54400 matches. 
Data[0][0]

In [ ]:
# Team 0’s hero IDs of 54400 matches.
Data[1][0]

In [ ]:
# Team 1’s stats of 54400 matches.
Data[2][0]

In [ ]:
#Team 0’s stats of 54400 matches.
Data[3][0]


In [ ]:
# Match result of 54400 matches.
# Format (Team ID that won the game, duration of the game)
Data[4][0]

In [ ]:
def to_categorical(n, num_classes: Optional[int] = None) -> np.ndarray:
    """Encode to categorical data."""
    if not (type(n) is np.ndarray):
        if type(n) is list:
            n = np.array(n)
        else:
            n = np.array([n])
    if len(n.shape) > 1:
        raise ValueError("To many dimensions")
    n = n.astype(np.int)
    if num_classes is None:
        num_classes = np.max(n) + 1
    encoded = np.zeros([len(n), num_classes])
    encoded[np.arange(len(n)), n] = 1
    return encoded


In [ ]:
np.unique(np.array(Data[1]).flatten()).shape

In [ ]:
hero_ids = np.unique(np.array(Data[1]).flatten())

In [ ]:
np.max(np.array(Data[1]).flatten())

In [ ]:
num_classes = np.max(np.array(Data[1]).flatten())+1

In [ ]:
def encode_teams(team1,team2):
    ret = np.zeros([1, num_classes])
    for e1 in team1:
        ret += to_categorical(e1, num_classes)
    for e2 in team2 :
        ret -= to_categorical(e2, num_classes)
    return ret
encode_teams(Data[0][0], Data[1][0])

In [ ]:
inputs = np.concatenate([encode_teams(Data[0][i], Data[1][i]) for i in range(len(Data[0]))], axis=0)

In [ ]:
outputs = np.array([Data[4][i][0] for i in range(len(Data[0]))])

In [ ]:
ndata = len(inputs)
nvalid = ndata//10
indexes = np.random.permutation(ndata)
validation_indexes = indexes[:nvalid]
train_indexes = indexes[nvalid:]

In [ ]:
inputs_train = inputs[train_indexes]
inputs_valid = inputs[validation_indexes]
outputs_train = outputs[train_indexes]
outputs_valid = outputs[validation_indexes]

In [ ]:
inputs_train.shape

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import (Activation, BatchNormalization, Dense,
                                     Input, Lambda, LeakyReLU, Reshape)
from tensorflow.keras.models import Model

In [ ]:

#     def mlp2_block(self, x, neurons: int, outshape: int):
#         """Two linear layers multilayer perceptron."""
#         x = Dense(neurons)(x)
#         x = BatchNormalization()(x)
#         x = LeakyReLU()(x)
#         x = Dense(outshape)(x)
#         x = BatchNormalization()(x)
#         x = LeakyReLU()(x)

#         return x

neurons = 256
input_h = Input(shape=(120))

x = Dense(neurons)(input_h)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dense(neurons)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dense(1)(x)
out_f = Activation("sigmoid")(x)

model = Model(inputs=input_h, outputs=out_f)


In [ ]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mae", "acc"])  # Is mse the right loss?
model.summary()

In [ ]:
# # Tensorboard
# tfdir = PATH_SUMMARIES() / Path(name)

# safe_rm(tfdir)
# tfdir.mkdir(parents=True, exist_ok=False)
# tbCallBack = TensorBoard(log_dir=str(tfdir), histogram_freq=0, write_graph=True, write_images=True, update_freq=128)

# # Save Callback
# cpdir = PATH_CHECKPOINTS() / Path(name)
# safe_rm(cpdir)
# cpdir.mkdir(parents=True, exist_ok=False)

# svCallBack = ModelCheckpoint(str(cpdir / "model-{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}.hdf5"), monitor="val_acc", verbose=0, save_best_only=False, save_weights_only=False, mode="auto", period=1)

In [ ]:
# history = model.fit(q_gen_train, steps_per_epoch=4*1024, epochs=epochs, validation_data=q_gen_valid, validation_steps=256, callbacks=[tbCallBack, svCallBack])
history = model.fit(x = inputs_train, y = outputs_train, batch_size=64, validation_data=(inputs_valid, outputs_valid), epochs=10)


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
y_true = outputs_valid
y_scores = res = model.predict(x = inputs_valid, batch_size=64 )
roc_auc = roc_auc_score(y_true, y_scores)
fpr, tpr, threshold = roc_curve(y_true, y_scores)
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

# Let us add more output

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
game_time = np.array([Data[4][i][1] for i in range(len(Data[0]))])
game_time_std = np.std(game_time)
game_time_mean = np.mean(game_time)
game_time = (game_time-game_time_mean)/game_time_std

In [ ]:
plt.hist(game_time, 100);

In [ ]:
outputs2 = np.array([outputs, game_time]).T
outputs_train = outputs2[train_indexes]
outputs_valid = outputs2[validation_indexes]

In [ ]:
outputs2.shape

In [ ]:
inputs.shape

In [ ]:
neurons = 256
input_h = Input(shape=(120))

x = Dense(neurons)(input_h)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dense(neurons)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
out_f = Dense(2)(x)
# out_f = Activation("sigmoid")(x)

model = Model(inputs=input_h, outputs=out_f)


In [ ]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mae", "acc"])  # Is mse the right loss?


In [ ]:
history = model.fit(x = inputs_train, y = outputs_train, batch_size=64, validation_data=(inputs_valid, outputs_valid), epochs=5)


In [ ]:
res = model.predict(x = inputs_valid, batch_size=64 )

In [ ]:
acc = np.mean(np.abs(res[:,0]>0.5 - outputs_valid[:,0]))
acc

In [ ]:
t = np.mean(np.abs(res[:,1] - outputs_valid[:,1]))
t*game_time_std/60, t

In [ ]:
1 - np.mean(np.abs(1*(res[:,0]>0.5) - outputs_valid[:,0]))

In [ ]:
1-np.mean(np.logical_xor(res[:,0]>0.5, outputs_valid[:,0]>0))

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
y_true = outputs_valid[:,0]
y_scores = res[:,0]
roc_auc = roc_auc_score(y_true, y_scores)
fpr, tpr, threshold = roc_curve(y_true, y_scores)
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

# Model using everything

In [ ]:
def encode_teams(team1,team2, st=0):
    ret = np.zeros([20, num_classes])
    for i, (e1, e2) in enumerate(zip(team1, team2)):
        r1 = to_categorical(e1, num_classes)
        r2 = to_categorical(e2, num_classes)
        if i==0:
            ret[4*i+0,:] = r1
            ret[4*i+1,:] = r2
        else:
            ret[4*i+0,:] = ret[4*(i-1)+2,:] + r1
            ret[4*i+1,:] = ret[4*(i-1)+3,:] + r2
        ret[4*i+2,:] = ret[4*i+0,:] - r2
        ret[4*i+3,:] = ret[4*i+1,:] - r1
    return ret[ 4*st:]


In [ ]:
test = encode_teams(Data[0][0], Data[1][0])
test.shape

In [ ]:
plt.imshow(encode_teams(np.arange(5), np.arange(5)+5))

In [ ]:
st = 0
inputs = np.concatenate([encode_teams(Data[0][i], Data[1][i], st) for i in range(len(Data[0]))], axis=0)

In [ ]:
t = np.mod(np.arange(20-st*4), 2)
outputs = np.concatenate([np.abs(t-Data[4][i][0]) for i in range(len(Data[0]))])

In [ ]:
outputs

In [ ]:
inputs.shape

In [ ]:
outputs.shape

In [ ]:
ndata = len(inputs)
nvalid = ndata//10
indexes = np.random.permutation(ndata)
validation_indexes = indexes[:nvalid]
train_indexes = indexes[nvalid:]

In [ ]:
inputs_train = inputs[train_indexes]
inputs_valid = inputs[validation_indexes]
outputs_train = outputs[train_indexes]
outputs_valid = outputs[validation_indexes]

In [ ]:
# def data_gen(t1, t2, outs)
#     st = 4
#     for team1, team2, out in zip(t1,t2, outs):
#         ret = encode_teams(team1, team2)
#         t = -(np.mod(np.arange(20-4*st), 2)*2-1) * out[0]
#         outs

In [ ]:
neurons = 256
input_h = Input(shape=(120))

x = Dense(neurons)(input_h)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dense(neurons)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dense(1)(x)
out_f = Activation("sigmoid")(x)

model = Model(inputs=input_h, outputs=out_f)


In [ ]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mae", "acc"])  # Is mse the right loss?


In [ ]:
history = model.fit(x = inputs_train, y = outputs_train, batch_size=512, shuffle=True, validation_data=(inputs_valid, outputs_valid), epochs=5)

In [ ]:
masks = []
num_pick = np.arange(1,11)
num_pick_input = np.sum(np.abs(inputs_valid), axis=1)
for i in num_pick:
    mask = num_pick_input==i
    masks.append(mask)
    if np.sum(mask):
        res = model.evaluate(x = inputs_valid[mask], y=outputs_valid[mask], batch_size=512,verbose=0)
        print(i, res[-1])
        res = model.predict(x = inputs_valid[mask], batch_size=512,verbose=0)
        score, ideal_score, nb_matches = compute_calibration_from_pred(res.flatten(), outputs_valid[mask])
        plt.figure()
        x = np.arange(10)/20+0.55
        plt.plot(x, score, "x-")
        plt.plot(x, ideal_score,  "o-")

In [ ]:
def compute_calibration_from_pred(
    p: np.array, target: np.array, nbins: int = 10
) -> Tuple[np.array, np.array, np.array]:
    """Compute calibration from prediction."""
    correct = (1*(p>0.5) - target) == 0
    confidence = np.abs(p*2-1) / 2 + 0.5
    score = np.zeros([nbins])
    ideal_score = np.zeros([nbins])
    nb_matches = np.zeros([nbins])
    for i in range(nbins):
        low = 0.5 + i * 0.5 / nbins
        high = 0.5 + (i + 1) * 0.5 / nbins
        interval = np.logical_and(confidence <= high, confidence >= low)
        n_match = np.sum(interval)
        if n_match:
            score[i] = np.mean(correct[interval])
            ideal_score[i] = np.mean(confidence[interval])
        else:
            score[i] = np.nan
            ideal_score[i] = (low + high) / 2
        nb_matches[i] = n_match
    return score, ideal_score, nb_matches

In [ ]:
# confidence = np.abs(res.flatten()*2-1) / 2 + 0.5
# plt.hist(confidence)

In [ ]:
res = model.predict(x = inputs_valid, batch_size=512,verbose=0)
score, ideal_score, nb_matches = compute_calibration_from_pred(res.flatten(), outputs_valid)

# Make the next pick

In [ ]:
# 1. Take a real example
pick = 6
mask = masks[pick-1]
select_ind = np.random.randint(np.sum(mask))
sample = inputs_valid[mask][select_ind]
np.sum(np.abs(sample))

In [ ]:
# compute all possible probabilities
possible_samples = []
corresponding_ids = []
for i in hero_ids:
    if sample[i]==0:
        new_sample = sample.copy()
        if np.mod(np.sum(np.abs(sample)),2)==0:
            new_sample[i] = 1
        else:
            new_sample[i] = -1
        corresponding_ids.append(i)
        possible_samples.append(new_sample)

In [ ]:
prob = model.predict(x=np.array(possible_samples)).flatten()

In [ ]:
for i,p in zip(corresponding_ids, prob):
    print(i,p)